### (Solo per google colab: associare google drive al notebook e download dei moduli)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# necessari anche torch, numpy, sklearn e pandas
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 77.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 109.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 21.8 MB/s eta 0:00:00


# IMPORT

In [3]:
from transformers import (
    GPT2Tokenizer,
    DataCollatorForLanguageModeling,
    TextDataset,
    GPT2LMHeadModel,
    TrainingArguments,
    Trainer,
    pipeline)

import pandas as pd
import numpy as np  


# BLOCCHI DA ESEGUIRE IN TRAINING MODE

### LETTURA DEL DATASET E SPLIT IN TRIAN E TEST

In [5]:
# =============================(Solo per la prima run)==========================================================
from sklearn.model_selection import train_test_split


df = pd.read_csv('./category_short9-10_soloHeadline.csv')   # leggo i dati dal drive
df['headline']=df['headline'].astype("string")
df.info()

train,test = train_test_split(df)               # eseguo lo split e salvo

train.to_csv('./category_short9-10_soloHeadline_train.txt',index=False) # il sistema richiede i dati in formato txt
test.to_csv('./category_short9-10_soloHeadline_test.txt',index=False)   # perché la funzione di trian richiede che il dataset dosponga di una funzione __len__
# avrei potuto tenere i dati in formato csv se avessi aggiunto a mano una funzione di len

# ===============================================================================================================

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209527 entries, 0 to 209526
Data columns (total 1 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   headline  209521 non-null  string
dtypes: string(1)
memory usage: 1.6 MB


In [6]:
# leggo i file
train_path = './category_short9-10_soloHeadline_train.txt'
test_path = './category_short9-10_soloHeadline_test.txt'

In [7]:
# scarico il tokenizer di GPT2
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [8]:
# scarico il data collator, un oggetto che costruisce batch utilizzando una lista di elementi del dataset come input
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [16]:
# eseguo il setup del train e test set con l'oggetto TextDataset.
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=train_path,
    block_size=128)
     
test_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=test_path,
    block_size=128)

/usr/local/lib/python3.8/dist-packages/transformers/data/datasets/language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [17]:
# faccio una prova di stampa del tokenizer
print(tokenizer.decode(train_dataset[5]))

's Beautiful Mind
Saturday's Morning Email: Funnies Edition
Syrians Fight For Survival Amid Assad's Poison Gas Warfare
A Bipartisan Response To Charlottesville And The Trump Administration
San Francisco Radio Stations Ban Lorde's 'Royals' Until After World Series
Bill Maher Is Sick Of Donald Trump Acting Like A 'Whiny Little Bitch'
What Makes You...You?
McCain’s Health Battle Casts Ugly Glare On GOP Health Care Assault
Nike Settles Lawsuit With Company Behind Lil Nas X's 'Satan Shoes'
What I Learned About Business From Making Art



### FINE TUNING

In [9]:
model = GPT2LMHeadModel.from_pretrained('gpt2')

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

In [19]:
training_args = TrainingArguments(
    output_dir = 'data/out', # la directory di output per le predictions ed i checkpoints (su colab questo è il path temporaneo della sesisone(50Gb) in modo da non saturare lo spazio del drive(15Gb))
    overwrite_output_dir = True, # sovrascrivere la cartella di output con i risultati
    per_device_train_batch_size = 32, # la batchsize del training
    per_device_eval_batch_size = 32, # la batchsize dell'evaluation. Queste scelte sono state prese a fronte di varie prove effettuate ed i risultati sono accettabili.
    learning_rate = 5e-5, # defaults a 5e-5, non è stato necessario cambiare
    num_train_epochs = 3, # numero totale delle epoche da eeguire durante il training
)

trainer = Trainer(
    model = model,
    args = training_args,
    data_collator=data_collator,
    train_dataset = train_dataset,
    eval_dataset = test_dataset
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [20]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 18310
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1719
  Number of trainable parameters = 124439808


Step,Training Loss
500,4.129200
1000,3.917600
1500,3.835400


Saving model checkpoint to data/out/checkpoint-500
Configuration saved in data/out/checkpoint-500/config.json
Model weights saved in data/out/checkpoint-500/pytorch_model.bin
Saving model checkpoint to data/out/checkpoint-1000
Configuration saved in data/out/checkpoint-1000/config.json
Model weights saved in data/out/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to data/out/checkpoint-1500
Configuration saved in data/out/checkpoint-1500/config.json
Model weights saved in data/out/checkpoint-1500/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1719, training_loss=3.9413309413517124, metrics={'train_runtime': 1929.7873, 'train_samples_per_second': 28.464, 'train_steps_per_second': 0.891, 'total_flos': 3588192829440000.0, 'train_loss': 3.9413309413517124, 'epoch': 3.0})

# ----------------------------------------------------------------------------------------------------

In [21]:
# salvo il modello. Il nome è temporaneo e da cambiare ogni volta per evitare di consfondersi o sovrascrivere per sbaglio
import torch
torch.save(model.state_dict(), "./<NOME_FILE>.pth")

In [ ]:
#salvare i checkpoint, o meglio spostare l'ultimo checkpoint in una cartella persistente del drive

!mv "/content/data/out/checkpoint-2000" "./salvataggi"

# ------ OPPURE -----------
# shutil.move("path/to/current/file", "path/to/new/destination/for/file")

# ----------------------------------------------------------------------------------------------------

### GENERAZIONE DEL TESTO

#### Generazione con il metodo STANDARD

In [22]:
generator = pipeline('text-generation', tokenizer='gpt2', model=model)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-g

In [ ]:
print(generator('Today is the day where', max_length=40)[0]['generated_text'])

#### Generazione con il metodo BEAM

Si mantengono in memoria le num_beam sequenze con la maggiore probabilità per ogni step. Questo riduce il rischio di perdersi sequenze di parole nascoste ad alta probabilità. Infine si sceglie quella con la probabilità maggiore.


In [ ]:
text_beam = generator('Today is the day where',
                      max_length=40,
                      num_beams=5)
print(text_beam[0]['generated_text'])

#### Generazione con il metodo RANDOM SAMPLING
la prossima parola è seleionata randomicamente in base alla distribuzione di probabilità condizionata dalle parole precedenti.

In [13]:
text_random_sampling = generator('Today is the day where',
                                 max_length=40,
                                 top_k=0,
                                 do_sample=True,
                                 temperature=0.7)
print(text_random_sampling[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Today is the day where his team would move on from Bob Odenkirk's injury and move on from the 2-2-2 that they were in when they lost the season opener to the


#### Generazione con il metodo TOP-K

La prossima parola è selezionata randomicamente tra le k parole che hanno distribuzione di probabilità condizionata maggiore di essere le parole successive.

{ex: si sceglie tra le K parole piu probabili}

In [14]:
text_k_sampling = generator('Today is the day where',
                            max_length=40,
                            top_k=40,
                            do_sample=True)
print(text_k_sampling[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Today is the day where we can start again. If you're a fan of Dragon Ball Z: Battle Z, please consider donating to the Project Gutenberg Literary Archive Initiative, because the donation is in no


#### Generazione con il metodo TOP-P

La selezione della parola successiva avviene randomicamente tra le parole del più piccolo set possibile in cui la probabilità cumulativa eccede la probabilità p.

{ex: si sceglie tra le parole più probabili che complessivamente mi dano più del P% di probabilità di essere le successive}

In [15]:
text_p_sampling = generator('Today is the day where',
                            max_length=40,
                            top_k=0,
                            top_p=0.92,
                            do_sample=True)
print(text_p_sampling[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Today is the day where the World ceases to be a country and starts to be a country of business, of agriculture and of commodities, which as they stand now, has the final teeth. We feel


# BLOCCHI DA ESEGUIRE IN INFERENCE MODE

In [4]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [5]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [6]:
model = GPT2LMHeadModel.from_pretrained('gpt2')

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

In [7]:
import torch
model.load_state_dict(torch.load("/content/drive/MyDrive/Progetti/modelli/salvataggi/DifferentiModelli-splittedSet-18-01.pth"))

<All keys matched successfully>

In [8]:
generator = pipeline('text-generation', tokenizer='gpt2', model=model)

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [9]:
print(generator('Today is the day where', max_length=40)[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Today is the day where the Internet's Going to get on board and make A Fight Back on the Internet"
"Here Are The Top 10 Books Of 2015, Ranked By Their Best Beauty Stories"


In [10]:
text_beam = generator('Today is the day where',
                      max_length=40,
                      num_beams=5)
print(text_beam[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Today is the day where I'm in the U.S. and I'm in the U.S.A."
Trump’s Pick For U.S. Ambassador To Israel Is A


In [11]:
text_random_sampling = generator('Today is the day where',
                                 max_length=40,
                                 top_k=0,
                                 do_sample=True,
                                 temperature=0.7)
print(text_random_sampling[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Today is the day where Americans, as a State, Can Choose to End the War on Drugs"
"Life Coach: Darlene Stewart, Parent, Artist, And Founder Of The New York


In [12]:
text_k_sampling = generator('Today is the day where',
                            max_length=40,
                            top_k=40,
                            do_sample=True)
print(text_k_sampling[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Today is the day where A 'First' Abortion Is Possible"
A Look at Life and the Struggle for Happiness
"HuffPost Rise: What You Need To Know, April 16 - April


In [13]:
text_p_sampling = generator('Today is the day where',
                            max_length=40,
                            top_k=0,
                            top_p=0.92,
                            do_sample=True)
print(text_p_sampling[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Today is the day where the Government Proposes To Save Your Life"""
SKEPTICS: The Evolution of Our Inner Child
4 Ways Buying A Pet Can Last Forever
Surprise!
